In [15]:
from ultralytics import YOLO
from pathlib import Path
import torch
import cv2
import time
import numpy as np
from shapely.geometry import Polygon
import asyncio
import tensorrt as trt

In [16]:
# Try to load model
if Path("yolov8x.pt").exists():
    model = YOLO("yolov8x.pt")

else:
    # Build from YAML and transfer weights
    model = YOLO("yolov8x.yaml").load("yolov8x.pt")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 80, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(400, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_r

In [17]:
# Convert to tensorRT
# model.export(
#     format="engine",
# )

model = YOLO("yolov8x.engine")

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [18]:
def onMouseClick(event, x, y, flags, param):
	global points

	if event == cv2.EVENT_LBUTTONDOWN:
		print(x, y)
		points.append((x, y))

def draw_overlap(boxes, polygon_points, frame):
	# Not enough points to form a polygon
	if len(polygon_points) < 3:
		return False
	
	user_polygon = np.array(polygon_points, np.int32)
	user_poly_shape = Polygon(user_polygon)

	is_overlap = False
	
	for box in boxes:
		box_polygon = np.array([
			(box[0], box[1]),
			(box[2], box[1]),
			(box[2], box[3]),
			(box[0], box[3]),
		], np.int32)
		
		if Polygon(box_polygon).intersects(user_poly_shape):
			# Draw red lines on the points of the overlapping box
			cv2.polylines(frame, [box_polygon], True, (0, 0, 255), 2)
			is_overlap = True
	
	return is_overlap

async def process_frame(frame):
	small_frame = cv2.resize(frame, (640, 480))
	results = await asyncio.to_thread(model, source=small_frame, conf=0.3, save=False, classes=[0], verbose=False)
	return results[0]


In [19]:
async def main():
	global points

	# Testing with youtube videos
	# -----------------------
	source = cv2.VideoCapture("./HD CCTV Camera video 3MP 4MP iProx CCTV HDCCTVCameras.net retail store.mp4")
	# -----------------------

	# source = cv2.VideoCapture(0)
	cv2.namedWindow("YOLO Output")
	cv2.setMouseCallback("YOLO Output", onMouseClick)

	# Points of zone
	points = []

	# For FPS calculation
	new_frame_time = 0
	prev_frame_time = 0

	# Frame skipper
	frame_skip = 2
	frame_count = 0

	# font which we will be using to display FPS 
	font = cv2.FONT_HERSHEY_SIMPLEX

	# is_overlapping = False

	while True:
		ret, frame = source.read()
		# Detect objects every n frames
		if frame_count % frame_skip == 0:
			results = await process_frame(frame)
			frame_count = 0
		
		frame_count += 1
		frame = results.plot()

		# Add circles at point coords
		for i, point in enumerate(points):
			# Draw circle at point
			cv2.circle(frame, point, 5, (243, 211, 74), 2) 
			# Draw line to connect to next point
			if len(points) > 1:
				next_point = points[i+1] if i < len(points)-1 else points[0]
				# color = (0, 0, 255) if is_overlapping else (243, 211, 74) # Change to red on overlap
				cv2.line(frame, point, next_point, (243, 211, 74), 2)

		draw_overlap(results.boxes.xyxy.tolist(), points, frame)

		# time when we finish processing for this frame 
		new_frame_time = time.time()
	
		# Calculate FPS6
		fps = 1 / (new_frame_time-prev_frame_time) 
		prev_frame_time = new_frame_time 
		fps = "FPS: " + str(int(fps))
		cv2.putText(frame, fps, (frame.shape[1]-200, 60), font, 1.5, (0, 0, 0), 7, cv2.LINE_AA)
		cv2.putText(frame, fps, (frame.shape[1]-200, 60), font, 1.5, (255, 255, 255), 2, cv2.LINE_AA)
		
		# frame = cv2.resize(frame, (frame.shape[1], frame.shape[0]))
		cv2.imshow("YOLO Output", frame)
		
		if cv2.waitKey(10) & 0xFF == ord("q"):
			break

	source.release()
	cv2.destroyAllWindows()
	
await main()

Loading yolov8x.engine for TensorRT inference...
376 207
495 382
143 355
80 166
294 83


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
